In [1]:
import sys
sys.path.append("../.")

In [2]:
import pandas as pd
import Utils.Medic1Utils as ut
from Utils.Utils import checkDateTimeFormat, stripAndJoin

In [3]:
df = pd.read_pickle("MedicItaliaDataNotClean.pkl")

In [4]:
def cleanElement(element: str) -> tuple[str]:
    """
    This function cleans and formats a text element containing a consultation conversation,
    separating user and doctor messages.

    :param element: element to clean
    :type element: str
    :return: cleaned element
    """
    elementSplitted = element.split("\n")
    entireText = []
    for line in elementSplitted:
        if "Questo consulto ha ricevuto" in line:
            break
        if "👉🏻" not in line and "Nutricia Fortimel" not in line:
            entireText.append(line)

    domandaSplitted = []
    indexesToRemove = []
    for i, line in enumerate(entireText):
        if checkDateTimeFormat(line):
            break
        domandaSplitted.append(line)
        indexesToRemove.append(i)

    textSenzaDomanda = []
    for i in range(len(entireText)):
        if i not in indexesToRemove:
            textSenzaDomanda.append(entireText[i])

    textSenzaDomanda = textSenzaDomanda[1::]
    domanda = "\n".join(domandaSplitted[1::])
    posts = ut.divideInPosts(textSenzaDomanda, "[#")

    cleanedElement = ""

    cleanedElement += "###UTENTE###\n"
    cleanedElement += domanda.strip() + "\n"
    for post in posts:
        cleanedElement += "\n"
        if ut.isPostFromDoctor(post):
            cleanedElement += "###DOTTORE###\n"
            cleanedElement += stripAndJoin(post[4::]) + "\n"
        else:
            cleanedElement += "###UTENTE###\n"
            cleanedElement += stripAndJoin(post[2::]) + "\n"
    return domandaSplitted[0].strip(), cleanedElement

In [5]:
import re
import time

basePath = "./Data/"

def printTofile(element: str) -> None:
    cleanedElement = cleanElement(element)
    title = cleanedElement[0].strip().replace(' ', '_')
    with open(basePath + re.sub(r'(?u)[^-\w.]', '', title) + "_" + str(time.time()) + ".txt", 'w', encoding="utf-16") as f:
        f.write(cleanedElement[1])


In [6]:
df.applymap(printTofile)

C:\Users\nico2\AppData\Local\Temp\ipykernel_15284\743398642.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(printTofile)


,0
0,None
1,None
2,None
3,None
4,None
...,...
18824,None
18825,None
18826,None
18827,None
